# Extração dos dados das estações, e S4
___

Os dados de estações e S4 estão organizados em um servidor postgresql rodando na minha maquina local. Para permitir a utilização por outros, este notebook, além de extrair os dados do postgresql vai gerar um csv, contendo as informações sobre as estações brutas. Assim, como um dataframes pandas, um por estação, comprimido usando o formatado xz, o que forneceu a extensão "pkl.xz".

As estações válidas são aquelas que apresentam algum dado de S4, pois existem estações que não coletaram dado nenhum durante o período de aquisição.

Note assim, que este notebook não executará em um máquina onde os dados não estejam armazenados no postgresql.

In [1]:
import aacgmv2
import os
import sys
import psycopg2

import datetime as dt
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mpl_toolkits.basemap import Basemap
from itertools import chain
from tqdm import tqdm

%matplotlib inline

In [2]:
conn = psycopg2.connect(database="data-research",
                        user="data-research",
                        password="data-research",
                        host="localhost",
                        port=5432)

In [3]:
df_station = pd.read_sql_query('select * from station;', con=conn)
df_station.to_csv("./data/stations.csv")
df_station = df_station.drop(columns=["startoperation", "stopoperation", "alias", "network"])

In [4]:
query = '''select scintillation.eventdate, scintillation.s4 
           from scintillation inner join station on 
           scintillation.station = station.identificationstation 
           where station.identificationstation = '%s' 
           order by scintillation.eventdate; 
        '''
drop_aux = []
select_station_and_df_s4 = []
for i in tqdm(df_station.itertuples()):
    station = i.identificationstation
    df_aux = pd.read_sql_query(query % station, con=conn)
    if len(df_aux) == 0:
        drop_aux.append(station)
    else:
        df_aux.to_pickle("./data_s4/df_" + station + ".pkl.xz", compression='xz')

54it [2:51:04, 190.07s/it]


In [5]:
df_station = df_station[~df_station['identificationstation'].isin(drop_aux)]
df_station.to_pickle("./data/df_station.pkl")

In [6]:
df_station

,identificationstation,height,latitude,longitude,state,city
0,bov,69.180,2.833920,-60.695380,rr,boa vista
6,tfe,0.057,-3.180000,-64.440000,am,tefe
7,imp,134.720,-5.310000,-47.280000,ma,imperatriz
8,nta,0.000,-5.836162,-35.121000,rn,natal
10,pvh,1130.000,-8.490000,-63.550000,ro,porto velho
11,pln,434.340,-9.230000,-40.300000,pe,petrolina
13,afl,267.000,-9.900000,-55.900000,mt,alta floresta
14,ios,0.000,-14.470000,-39.100000,ba,ilheus
15,cub,278.000,-15.555200,-56.069800,mt,cuiaba
16,bsa,1050.000,-15.764200,-47.869400,df,brasilia
